# Neighborhood Cluster in Toronto by Venues

## Introduction
In this lab, neighborhoods in Toronto will be clustered by the frequency of venues. Firstly, the coordinates of the neighborhoods in Toronto will be collected. Then, the explore function will be used to get the most common venues in each neighborhood, and then use this feature to group the neighborhoods into clusters. The k-means clustering algorithm will be used to complete this task. Finally, Folium library will be used to visualize the neighborhood in Toronto and their emerging clusters.

## Table of Contents
<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Prepare Dataset</a>

2. <a href="#item2">Explore Neighborhoods in Toronto</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Before getting the data and start exploring it, download all the dependencies needed.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge geocoder --yes # uncomment this line if you haven't completed the Foursquare API lab
import geocoder

!conda install -c conda-forge wikipedia --yes
import wikipedia as wp

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    openssl-1.0.2p             |       h470a237_2         3.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0         conda-forge
    geopy:         1.18.1-py_0       conda-forge

The following packages will be UPDATED:

    openssl:       1.0.2p-h470a237_1 conda-forge --> 1.0.2p-h470a237_2 conda-forge


geopy-1.18.1         | 51 KB     | ##################################### | 100% 
ope

## 1. Download and Prepare Dataset

In [3]:
### Prepare borough and neighborhood list ###

# Get the table of neighborhoods list from wiki 
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html, header = 0)[0]

# Drope the rows with Borough of "Not assigned"
df_tt = df[df.Borough != 'Not assigned']
df_tt.reset_index(drop = True, inplace = True)

# Rename 'Neighborhood'
df_tt.rename(columns = {'Neighbourhood' : 'Neighborhood'}, inplace = True)

# Merge rows with the same Postcode
# df_tt = df_tt.groupby(['Postcode', 'Borough'], as_index=False).agg(lambda x: ', '.join(set(x)))

# Set 'Not assigned' Neigbourhood to be equal to Borough
na_nb_idx = df_tt['Neighborhood'] == 'Not assigned'
df_tt.at[na_nb_idx, 'Neighborhood'] = df_tt[na_nb_idx].Borough

# Keep the Borough with word 'Toronto' only
# df_tt = df_tt[df_tt['Borough'].str.contains('Toronto')]
# df_tt.reset_index(drop = True, inplace = True)

# Drop duplicated neighborhood
df_tt.drop_duplicates(subset = ['Neighborhood'], inplace = True)

# Show the shape of the df
print('The dataframe shape is: ', df_tt.shape)
df_tt.head()

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/frame.py:2587: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[index, col] = val

The dataframe shape is:  (210, 3)


/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [4]:
### Add latitude and longitude of each neighborhood ###
  
# Create Latitude and Longitude columns
df_tt['Latitude'] = np.nan
df_tt['Longitude'] = np.nan

for idx, nb in zip(df_tt.index, df_tt['Neighborhood']):
    geolocator = Nominatim()
    try:
        g = geolocator.geocode('{}, Toronto, Ontario'.format(nb))
        df_tt.at[idx, 'Latitude'] = g.latitude
        df_tt.at[idx, 'Longitude'] = g.longitude
    except:
        df_tt.at[idx, 'Latitude'] = 0
        df_tt.at[idx, 'Longitude'] = 0

df_tt = df_tt[df_tt['Latitude'] != 0]

# Display the results
df_tt.shape

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/n

(200, 5)

In [5]:
df_tt

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.761124,-79.324059
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
3,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457
4,M6A,North York,Lawrence Heights,43.722778,-79.450933
5,M6A,North York,Lawrence Manor,43.722079,-79.437507
6,M7A,Queen's Park,Queen's Park,43.659980,-79.390369
7,M9A,Etobicoke,Islington Avenue,43.764431,-79.574768
8,M1B,Scarborough,Rouge,43.804930,-79.165837
9,M1B,Scarborough,Malvern,43.809196,-79.221701


# 2. Explore Neighborhoods in Toronto

In [6]:
# Define Foursquare ID and secret
CLIENT_ID = 'UCBUUCWTJQ2OMUEJAEGOEA51MYEWYTANECB4GOCTC5D3C25I' # Foursquare ID
CLIENT_SECRET = 'FXBF0E4ZLSTETZNDROA0WSZ0TFMI4OQXXTM5MVMDZUNEO0DU' # Foursquare Secret
VERSION = '20190101' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UCBUUCWTJQ2OMUEJAEGOEA51MYEWYTANECB4GOCTC5D3C25I
CLIENT_SECRET:FXBF0E4ZLSTETZNDROA0WSZ0TFMI4OQXXTM5MVMDZUNEO0DU


In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500): # Function to get the nearby venues
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        jres = requests.get(url).json()
        filename = name + '.json'
        with open(filename, 'w') as outfile:
             json.dump(jres, outfile)        
        
        results = jres["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
# Explore the venues in each neighborhood
LIMIT = 100
neighborhood_venues = getNearbyVenues(df_tt['Neighborhood'], df_tt['Latitude'], df_tt['Longitude'], 500)
neighborhood_venues.shape

Parkwoods
Victoria Village
Harbourfront
Regent Park
Lawrence Heights
Lawrence Manor
Queen's Park
Islington Avenue
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
St. James Town
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
Downsview East
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition

(6242, 7)

In [9]:
neighborhood_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.761124,-79.324059,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.761124,-79.324059,Tim Hortons,43.760668,-79.326368,Café
2,Parkwoods,43.761124,-79.324059,A&W Canada,43.760643,-79.326865,Fast Food Restaurant
3,Parkwoods,43.761124,-79.324059,Food Basics,43.760865,-79.326015,Supermarket
4,Parkwoods,43.761124,-79.324059,Shoppers Drug Mart,43.760857,-79.324961,Pharmacy


In [10]:
print('There are {} uniques categories.'.format(len(neighborhood_venues['Venue Category'].unique())))

There are 337 uniques categories.


# 3. Analyze Each Neighborhood

In [11]:
## Construct encoded data
# one hot encoding
neighborhood_onehot = pd.get_dummies(neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add Neighborhood column to dataframe
neighborhood_onehot['Neighborhood'] = neighborhood_venues['Neighborhood']

# move Neighborhood column to the first column
cols = neighborhood_onehot.columns.tolist()
cols.insert(0, cols.pop(cols.index('Neighborhood')))
neighborhood_onehot = neighborhood_onehot.reindex(columns= cols)

neighborhood_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Plane,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Road,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shi

In [12]:
# Goup rows by Neighborhood and by taking the mean of the frequency of occurrence of each category
neighborhood_grouped = neighborhood_onehot.groupby('Neighborhood').mean().reset_index()

neighborhood_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Plane,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Road,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shi

In [98]:
# num_top_venues = 10

# for pcd in neighborhood_grouped['Neighborhood']:
#    print("----"+pcd+"----")
#    temp = neighborhood_grouped[neighborhood_grouped['Neighborhood'] == pcd].T.reset_index()
#    temp.columns = ['venues','freq']
#    temp = temp.iloc[1:]
#    temp['freq'] = temp['freq'].astype(float)
#    temp = temp.round({'freq': 2})
#    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
#    print('\n')

In [13]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [14]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Restaurant'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Restaurant'.format(ind+1))

# create a new dataframe
neighborhood_sorted = pd.DataFrame(columns=columns)
neighborhood_sorted['Neighborhood'] = neighborhood_grouped['Neighborhood']

for ind in np.arange(neighborhood_grouped.shape[0]):
    neighborhood_sorted.iloc[ind, 1:] = return_most_common_venues(neighborhood_grouped.iloc[ind, :], num_top_venues)

neighborhood_sorted

,Neighborhood,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant,10th Most Common Restaurant
0,Adelaide,Coffee Shop,Hotel,Café,Gastropub,Restaurant,Clothing Store,American Restaurant,Italian Restaurant,Seafood Restaurant,Japanese Restaurant
1,Agincourt,Chinese Restaurant,Korean Restaurant,Vietnamese Restaurant,Asian Restaurant,Rental Car Location,Butcher,Train Station,Coffee Shop,Hong Kong Restaurant,Cantonese Restaurant
2,Agincourt North,Chinese Restaurant,Ice Cream Shop,Bakery,Fast Food Restaurant,Pharmacy,Sporting Goods Shop,Juice Bar,Gift Shop,Taco Place,Bank
3,Albion Gardens,Coffee Shop,Café,Deli / Bodega,Hotel,Restaurant,Italian Restaurant,Aquarium,American Restaurant,Seafood Restaurant,Scenic Lookout
4,Alderwood,Pizza Place,Sandwich Place,Pharmacy,Gym,Pub,Coffee Shop,Skating Rink,Zoo,Ethiopian Restaurant,Drugstore
5,Bathurst Manor,Playground,Convenience Store,Park,Baseball Field,Falafel Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
6,Bathurst Quay,Coffee Shop,Café,Park,Diner,Pub,Caribbean Restaurant,Garden,Grocery Store,Gym,Sculpture Garden
7,Bayview Village,Pizza Place,Fast Food Restaurant,Bank,Outdoor Supply Store,Hardware Store,Gas Station,Coffee Shop,Sandwich Place,Sporting Goods Shop,Breakfast Spot
8,Berczy Park,Coffee Shop,Café,Restaurant,Italian Restaurant,Hotel,Cocktail Bar,Bakery,Japanese Restaurant,Seafood Restaurant,Gym
9,Birch Cliff,Park,Gym,Gym Pool,Business Service,Drugstore,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant


# 4. Cluster FSAs

In [34]:
# set number of clusters
kclusters = 5

neighborhood_grouped_clustering = neighborhood_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighborhood_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2,
       1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2,
       1, 1, 1, 2, 1, 1, 2, 1, 1, 0, 1, 1, 1, 2, 2, 1, 1, 0, 0, 1, 2, 2,
       1, 2, 1, 1, 0, 2, 2, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 0, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 1, 3, 1, 1, 1, 3, 3, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 1, 1],
      dtype=int32)

In [35]:
neighborhood_merged = neighborhood_sorted

# add clustering labels
neighborhood_merged['Cluster Labels'] = kmeans.labels_

neighborhood_merged['Latitude'] = np.nan
neighborhood_merged['Longitude'] = np.nan

# For each Neighborhood, find corresponding coordinates in df_tt
for idx in neighborhood_merged.index.values:
    cord_idx = df_tt['Neighborhood'] == neighborhood_merged.loc[idx, 'Neighborhood']
    neighborhood_merged.at[idx, 'Latitude'] = df_tt.loc[cord_idx, 'Latitude'].values
    neighborhood_merged.at[idx, 'Longitude'] = df_tt.loc[cord_idx, 'Longitude'].values

neighborhood_merged.head()

,Neighborhood,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant,10th Most Common Restaurant,Cluster Labels,Latitude,Longitude
0,Adelaide,Coffee Shop,Hotel,Café,Gastropub,Restaurant,Clothing Store,American Restaurant,Italian Restaurant,Seafood Restaurant,Japanese Restaurant,1,43.650528,-79.379515
1,Agincourt,Chinese Restaurant,Korean Restaurant,Vietnamese Restaurant,Asian Restaurant,Rental Car Location,Butcher,Train Station,Coffee Shop,Hong Kong Restaurant,Cantonese Restaurant,1,43.785353,-79.278549
2,Agincourt North,Chinese Restaurant,Ice Cream Shop,Bakery,Fast Food Restaurant,Pharmacy,Sporting Goods Shop,Juice Bar,Gift Shop,Taco Place,Bank,1,43.808038,-79.266439
3,Albion Gardens,Coffee Shop,Café,Deli / Bodega,Hotel,Restaurant,Italian Restaurant,Aquarium,American Restaurant,Seafood Restaurant,Scenic Lookout,1,43.644903,-79.381836
4,Alderwood,Pizza Place,Sandwich Place,Pharmacy,Gym,Pub,Coffee Shop,Skating Rink,Zoo,Ethiopian Restaurant,Drugstore,1,43.601717,-79.545232


In [36]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhood_merged['Latitude'], neighborhood_merged['Longitude'], neighborhood_merged['Neighborhood'], neighborhood_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
       
map_clusters

# 5. Exame Clusters

## Cluster 0

In [38]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 0, neighborhood_merged.columns[[0] + [11] + list(range(1, 10))]]

,Neighborhood,Cluster Labels,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant
19,Clairlea,0,Fast Food Restaurant,Sandwich Place,Convenience Store,Grocery Store,Intersection,Pharmacy,Diner,Event Space,Dumpling Restaurant
21,Cliffcrest,0,Fast Food Restaurant,Wings Joint,Pizza Place,Discount Store,Furniture / Home Store,Liquor Store,Burger Joint,Zoo,Falafel Restaurant
42,Emery,0,Fast Food Restaurant,Sandwich Place,Tea Room,Caribbean Restaurant,Vietnamese Restaurant,Coffee Shop,Ethiopian Restaurant,Drugstore,Dumpling Restaurant
75,Kennedy Park,0,Fast Food Restaurant,Asian Restaurant,Grocery Store,Zoo,Falafel Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
83,L'Amoreaux East,0,Fast Food Restaurant,Chinese Restaurant,Japanese Restaurant,Grocery Store,Nail Salon,Coffee Shop,Sandwich Place,Pharmacy,Pizza Place
84,L'Amoreaux West,0,Fast Food Restaurant,Chinese Restaurant,Japanese Restaurant,Grocery Store,Nail Salon,Coffee Shop,Sandwich Place,Pharmacy,Pizza Place
92,Malvern,0,Pizza Place,Pharmacy,Park,Sandwich Place,Fast Food Restaurant,Grocery Store,Bubble Tea Shop,Skating Rink,Ethiopian Restaurant
99,Mimico NE,0,Sandwich Place,Fast Food Restaurant,Bus Stop,American Restaurant,Flea Market,Fish Market,Eastern European Restaurant,Egyptian Restaurant,Electronics Store
100,Mimico NW,0,Sandwich Place,Fast Food Restaurant,Bus Stop,American Restaurant,Flea Market,Fish Market,Eastern European Restaurant,Egyptian Restaurant,Electronics Store
101,Mimico South,0,Sandwich Place,Fast Food Restaurant,Bus Stop,American Restaurant,Flea Market,Fish Market,Eastern European Restaurant,Egyptian Restaurant,Electronics Store


## Cluster 1

In [39]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 1, neighborhood_merged.columns[[0] + [11] + list(range(1, 10))]]

,Neighborhood,Cluster Labels,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant
0,Adelaide,1,Coffee Shop,Hotel,Café,Gastropub,Restaurant,Clothing Store,American Restaurant,Italian Restaurant,Seafood Restaurant
1,Agincourt,1,Chinese Restaurant,Korean Restaurant,Vietnamese Restaurant,Asian Restaurant,Rental Car Location,Butcher,Train Station,Coffee Shop,Hong Kong Restaurant
2,Agincourt North,1,Chinese Restaurant,Ice Cream Shop,Bakery,Fast Food Restaurant,Pharmacy,Sporting Goods Shop,Juice Bar,Gift Shop,Taco Place
3,Albion Gardens,1,Coffee Shop,Café,Deli / Bodega,Hotel,Restaurant,Italian Restaurant,Aquarium,American Restaurant,Seafood Restaurant
4,Alderwood,1,Pizza Place,Sandwich Place,Pharmacy,Gym,Pub,Coffee Shop,Skating Rink,Zoo,Ethiopian Restaurant
6,Bathurst Quay,1,Coffee Shop,Café,Park,Diner,Pub,Caribbean Restaurant,Garden,Grocery Store,Gym
7,Bayview Village,1,Pizza Place,Fast Food Restaurant,Bank,Outdoor Supply Store,Hardware Store,Gas Station,Coffee Shop,Sandwich Place,Sporting Goods Shop
8,Berczy Park,1,Coffee Shop,Café,Restaurant,Italian Restaurant,Hotel,Cocktail Bar,Bakery,Japanese Restaurant,Seafood Restaurant
10,Bloordale Gardens,1,Convenience Store,Donut Shop,Sandwich Place,Intersection,Shipping Store,Coffee Shop,Event Space,Dumpling Restaurant,Eastern European Restaurant
11,Brockton,1,Bar,Park,Café,Vietnamese Restaurant,South American Restaurant,Jazz Club,Portuguese Restaurant,Coffee Shop,Dive Bar


## Cluster 2

In [40]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 2, neighborhood_merged.columns[[0] + [11] + list(range(1, 10))]]

,Neighborhood,Cluster Labels,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant
5,Bathurst Manor,2,Playground,Convenience Store,Park,Baseball Field,Falafel Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
9,Birch Cliff,2,Park,Gym,Gym Pool,Business Service,Drugstore,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant
20,Clarks Corners,2,Park,Convenience Store,Caribbean Restaurant,Drugstore,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
43,Eringate,2,Park,Zoo,Falafel Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
48,Forest Hill North,2,Playground,Bank,Park,Drugstore,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
49,Forest Hill SE,2,Playground,Bank,Park,Drugstore,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
50,Forest Hill West,2,Playground,Bank,Park,Drugstore,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant
60,Henry Farm,2,Tennis Court,Lawyer,Restaurant,Park,Zoo,Event Space,Eastern European Restaurant,Egyptian Restaurant,Electronics Store
64,Humber Bay,2,Construction & Landscaping,Pool,Harbor / Marina,Park,Exhibit,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store
65,Humber Summit,2,Construction & Landscaping,Empanada Restaurant,Bakery,Park,Dumpling Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space


## Cluster 3

In [41]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 3, neighborhood_merged.columns[[0] + [11] + list(range(1, 10))]]

,Neighborhood,Cluster Labels,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant
147,South Steeles,3,Playground,Zoo,Farm,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
151,Steeles East,3,Playground,Zoo,Farm,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
152,Steeles West,3,Playground,Zoo,Farm,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space


## Cluster 4

In [42]:
neighborhood_merged.loc[neighborhood_merged['Cluster Labels'] == 4, neighborhood_merged.columns[[0] + [11] + list(range(1, 10))]]

,Neighborhood,Cluster Labels,1st Most Common Restaurant,2nd Most Common Restaurant,3rd Most Common Restaurant,4th Most Common Restaurant,5th Most Common Restaurant,6th Most Common Restaurant,7th Most Common Restaurant,8th Most Common Restaurant,9th Most Common Restaurant
193,York Mills,4,Wine Shop,Zoo,Farm,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
194,York Mills West,4,Wine Shop,Zoo,Farm,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
